In [63]:
import pandas
import numpy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb
from sklearn.metrics import roc_auc_score

import pickle
from sklearn.externals import joblib
pd.set_option('display.max_columns', 500)

In [64]:
id_281 = [6391, 6399, 6406, 6418, 6422, 6458, 6477, 6486, 6504, 6510, 6528, 6529, 6541, 6564, 6589, 6592, 6596, 6599, 6601, 6604, 6619, 6678, 6712, 6729, 6756, 6763, 6769, 6773, 6783, 6789, 6812, 6825, 6828, 6833, 6844, 6863, 6865, 6874, 6903, 6921, 6947, 6953, 6969, 6983, 6990, 7004, 7005, 7011, 7032, 7039, 7044, 7076, 7080, 7091, 7103, 7114, 7120, 7133, 7143, 7157, 7169, 7193, 7196, 7204, 7218, 7230, 7244, 7285, 7289, 7296, 7318, 7325, 7340, 7364, 7376, 7383, 7396, 7400, 7463, 7468, 7482, 7504, 7517, 7525, 7538, 7543, 7546, 7569, 7574, 7580, 7597, 7598, 7636, 7647, 7649, 7723, 7757, 7761, 7767, 7775, 7784, 7800, 7811, 7818, 7824, 7825, 7833, 7861, 7914, 7937, 7965, 7985, 8016, 8020, 8049, 8053, 8056, 8058, 8061, 8070, 8077, 8090, 8099, 8144, 8161, 8163, 8164, 8179, 8196, 8208, 8232, 8246, 8258, 8288, 8289, 8297, 8306, 8322, 8326, 8344, 8376, 8399, 8402, 8404, 8410, 8412, 8465, 8472, 8476, 8492, 8557, 8589, 8606, 8622, 8627, 8658, 8660, 8662, 8673, 8678, 8715, 8722, 8724, 8751, 8777, 8790, 8810, 8811, 8821, 8829, 8835, 8841, 8852, 8878, 8884, 8895, 8905, 8908, 8910, 8955, 9006, 9011, 9013, 9041, 9057, 9074, 9092, 9130, 9156, 9173, 9182, 9189, 9225, 9244, 9246, 9251, 9256, 9258, 9259, 9264, 9276, 9284, 9306, 9311, 9314, 9323, 9329, 9365, 9372, 9390, 9396, 9401, 9420, 9430, 9449, 9453, 9460, 9474, 9483, 9489, 9500, 9514, 9556, 9558, 9586, 9599, 9602, 9612, 9618, 9665, 9669, 9678, 9709, 9721, 9738, 9777, 9791, 9797, 9802, 9807, 9832, 9842, 9867, 9868, 9881, 9892, 9905, 9932, 9940, 9952, 9953, 9955, 9984, 9990, 10010, 10019, 10033, 10035, 10045, 10047, 10050, 10052, 10054, 10071, 10086, 10091, 10097, 10121, 10127, 10141, 10159, 10200, 10202, 10233, 10244, 10262, 10264, 10266, 10277, 10289, 10298]

In [65]:
demo_df = pandas.read_csv('data/demo.csv', index_col='ip_id')
y_train = pandas.read_csv('data/y_train.csv', index_col='ip_id')

In [66]:
df_y_test = pd.read_csv('data/y_test_index.csv',index_col='ip_id')

In [67]:
df_test = demo_df[demo_df.index.isin(df_y_test.index)]
df_test.shape

(3938, 9)

In [68]:
df_test = df_test[df_test.index.isin(id_281)]
df_test.shape

(281, 9)

In [69]:
df_test = df_test[df_test.index.isin(pd.read_csv('data/cc_txn.csv')['ip_id'])==False]
df_test.shape

(281, 9)

In [70]:
demo_df.drop(demo_df.select_dtypes('object').columns.tolist(),axis=1,inplace=True)

In [71]:
demo_df['is_train'] = numpy.where(demo_df.index.isin(y_train.index),1,0)

In [72]:
def feature_select(df_all, thres = 0.5):
    res = 1
    while res > thres:
        columns = df_all.columns.tolist()

        y = df_all['is_train'].values
        X = df_all.drop(['is_train'],axis=1).values
        X_train, X_val, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=0)
        
        
        rfc = RandomForestClassifier(n_jobs=-1, max_depth=5, min_samples_leaf = 5,
                                    n_estimators=100)
        rfc.fit(X_train,y_train)
        res_prob = rfc.predict_proba(X_val)[:, 1]

        fpr, tpr, thresholds = metrics.roc_curve(y_test.ravel(), res_prob, pos_label=1)
        res = metrics.auc(fpr, tpr)
        
        if res > thres:
            df_all= df_all.drop([columns[rfc.feature_importances_.argmax()]],axis=1)
    
    print(res)
    return df_all

In [81]:
df_select = feature_select(demo_df,0.7)

0.5189661949685536


In [82]:
df_select.head()

,no_of_dpnd_chl,gnd_cd,is_train
ip_id,,,
1,0.0,2.0,1
2,1.0,2.0,1
3,0.0,2.0,1
4,0.0,2.0,1
5,1.0,2.0,1


In [36]:
df_train_features = demo_df[demo_df.index.isin(y_train.index)][[col for col in df_select.drop(['is_train'],axis=1).columns]]
df_train_label = y_train

In [39]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
clf1 = LogisticRegression(solver='lbfgs', multi_class='multinomial',
                          random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()
# X_train = df_train_features[:5000]
# y_train = df_train_label[:5000]
eclf1 = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')
eclf1 = eclf1.fit(df_train_features, df_train_label)
print(eclf1.predict(df_train_features))

import numpy as np
from sklearn.metrics import roc_auc_score
y_true = df_train_label
y_scores = eclf1.predict_proba(df_train_features)[:,1]
print (roc_auc_score(y_true, y_scores))

[0. 0. 0. ... 1. 0. 0.]
0.5679598124820779


/home/thanatlapthawan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/thanatlapthawan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/thanatlapthawan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [41]:
from keras.models import Model
from keras.layers import Input, Dense
import keras

Using TensorFlow backend.


In [44]:
X = df_train_features.values
print(X.shape)
Y = df_train_label.values
print(Y.shape)

(6361, 2)
(6361, 1)


In [45]:
inputs = Input(shape=(2,))
x = Dense(64, activation='relu')(inputs)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X, Y, batch_size=32, epochs=10)
y_true = df_train_label
y_scores = model.predict(X)[:,0]
print (roc_auc_score(y_true, y_scores))

Epoch 1/10
6361/6361 [==============================] - 1s 117us/step - loss: 0.2937 - acc: 0.9285
Epoch 2/10
6361/6361 [==============================] - 0s 69us/step - loss: 0.2361 - acc: 0.9362
Epoch 3/10
6361/6361 [==============================] - 0s 72us/step - loss: 0.2310 - acc: 0.9368
Epoch 4/10
6361/6361 [==============================] - 0s 75us/step - loss: 0.2273 - acc: 0.9377
Epoch 5/10
6361/6361 [==============================] - 0s 71us/step - loss: 0.2263 - acc: 0.9379
Epoch 6/10
6361/6361 [==============================] - 0s 75us/step - loss: 0.2254 - acc: 0.9382
Epoch 7/10
6361/6361 [==============================] - 0s 72us/step - loss: 0.2244 - acc: 0.9387
Epoch 8/10
6361/6361 [==============================] - 0s 75us/step - loss: 0.2242 - acc: 0.9382
Epoch 9/10
6361/6361 [==============================] - 0s 71us/step - loss: 0.2241 - acc: 0.9381
Epoch 10/10
6361/6361 [==============================] - 0s 66us/step - loss: 0.2232 - acc: 0.9392
0.6313843946811977

In [60]:
df_test = df_test[df_test.index.isin(df_y_test.index)][[col for col in df_select.drop(['is_train'],axis=1).columns]]
result = model.predict(df_test)[:,0]
df_test['prob1'] = result
df_test[['prob1']].to_csv('result_281_yortz.csv')

In [47]:
inputs = Input(shape=(2,))
x = Dense(64, activation='relu')(inputs)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X, Y, batch_size=32, epochs=10)
y_true = df_train_label
y_scores = model.predict(X)[:,0]
print (roc_auc_score(y_true, y_scores))

Epoch 1/10
6361/6361 [==============================] - 1s 119us/step - loss: 0.2667 - acc: 0.9349
Epoch 2/10
6361/6361 [==============================] - 0s 62us/step - loss: 0.2334 - acc: 0.9360
Epoch 3/10
6361/6361 [==============================] - 0s 74us/step - loss: 0.2291 - acc: 0.9370
Epoch 4/10
6361/6361 [==============================] - 0s 74us/step - loss: 0.2274 - acc: 0.9384
Epoch 5/10
6361/6361 [==============================] - 0s 74us/step - loss: 0.2250 - acc: 0.9384
Epoch 6/10
6361/6361 [==============================] - 0s 74us/step - loss: 0.2251 - acc: 0.9365
Epoch 7/10
6361/6361 [==============================] - 0s 74us/step - loss: 0.2242 - acc: 0.9379
Epoch 8/10
6361/6361 [==============================] - 0s 73us/step - loss: 0.2228 - acc: 0.9387
Epoch 9/10
6361/6361 [==============================] - 0s 68us/step - loss: 0.2244 - acc: 0.9381
Epoch 10/10
6361/6361 [==============================] - 1s 83us/step - loss: 0.2236 - acc: 0.9387
0.6313843946811977

In [49]:
inputs = Input(shape=(2,))
x = Dense(64, activation='relu')(inputs)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=predictions)
sgd = keras.optimizers.SGD(lr=0.01, clipnorm=1.)
model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X, Y, batch_size=32, epochs=10)
y_true = df_train_label
y_scores = model.predict(X)[:,0]
print (roc_auc_score(y_true, y_scores))

Epoch 1/10
6361/6361 [==============================] - 1s 124us/step - loss: 0.3549 - acc: 0.9311
Epoch 2/10
6361/6361 [==============================] - 0s 67us/step - loss: 0.2421 - acc: 0.9349
Epoch 3/10
6361/6361 [==============================] - 0s 71us/step - loss: 0.2385 - acc: 0.9349
Epoch 4/10
6361/6361 [==============================] - 0s 67us/step - loss: 0.2364 - acc: 0.9349
Epoch 5/10
6361/6361 [==============================] - 0s 69us/step - loss: 0.2351 - acc: 0.9349
Epoch 6/10
6361/6361 [==============================] - 0s 72us/step - loss: 0.2339 - acc: 0.9349
Epoch 7/10
6361/6361 [==============================] - 0s 69us/step - loss: 0.2330 - acc: 0.9349
Epoch 8/10
6361/6361 [==============================] - 0s 69us/step - loss: 0.2321 - acc: 0.9349
Epoch 9/10
6361/6361 [==============================] - 0s 69us/step - loss: 0.2315 - acc: 0.9349
Epoch 10/10
6361/6361 [==============================] - 0s 68us/step - loss: 0.2309 - acc: 0.9349
0.5783584302238209

In [51]:
inputs = Input(shape=(2,))
x = Dense(64, activation='relu')(inputs)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=predictions)
adam = keras.optimizers.Adam(lr=0.02, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X, Y, batch_size=32, epochs=10)
y_true = df_train_label
y_scores = model.predict(X)[:,0]
print (roc_auc_score(y_true, y_scores))

Epoch 1/10
6361/6361 [==============================] - 1s 178us/step - loss: 0.2446 - acc: 0.9368
Epoch 2/10
6361/6361 [==============================] - 1s 89us/step - loss: 0.2336 - acc: 0.9362
Epoch 3/10
6361/6361 [==============================] - 1s 84us/step - loss: 0.2299 - acc: 0.9374
Epoch 4/10
6361/6361 [==============================] - 1s 85us/step - loss: 0.2266 - acc: 0.9370
Epoch 5/10
6361/6361 [==============================] - 1s 84us/step - loss: 0.2263 - acc: 0.9390
Epoch 6/10
6361/6361 [==============================] - 1s 82us/step - loss: 0.2255 - acc: 0.9385
Epoch 7/10
6361/6361 [==============================] - 1s 86us/step - loss: 0.2252 - acc: 0.9381
Epoch 8/10
6361/6361 [==============================] - 1s 84us/step - loss: 0.2241 - acc: 0.9384
Epoch 9/10
6361/6361 [==============================] - 1s 88us/step - loss: 0.2243 - acc: 0.9385
Epoch 10/10
6361/6361 [==============================] - 1s 88us/step - loss: 0.2237 - acc: 0.9385
0.60845540600587
